In [1]:

from keras import backend as K
import matplotlib.pyplot as plt
from numpy import array
import numpy as np
import tensorflow.keras.layers as tfl
import tensorflow as tf
import tensorflow_datasets as tfds
import json
import os, time
from numpy import argmax

Using TensorFlow backend.


In [2]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [3]:
tf.__version__

'2.1.0'

In [4]:

tokenizer = tfds.features.text.Tokenizer(alphanum_only=False)
vocabulary_set = set()
site_words=125
all_genres=set()
max_entries=500
entries=0

In [5]:
genres_list=[]
sites_list=[]
for r, d, f in os.walk("./data"):
    for file in f:
        if '.json' in file:  # only load 10 jsons for now

            with open(os.path.join(r, file), mode='r', encoding='utf-8-sig') as prep_file:
                script_data = json.load(prep_file)
                genres=script_data["genres"] if script_data["genres"] else []
                if None in genres:
                    genres.remove(None)
                all_genres.update(genres)
                n=0
                script_tokenized=tokenizer.tokenize(script_data["script"])
                while (n+1)*site_words<len(script_tokenized):
                    site=script_tokenized[n:n+site_words]
                    vocabulary_set.update(site)
                    genres_list.append(genres)
                    sites_list.append(site)
                    entries+=1
                    if entries>max_entries:
                        break
                    n+=1
                
                if entries>max_entries:
                    break



In [6]:
sites_encoded=[]
site_text_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)
for site in sites_list:
    site_joined="".join(site)

    sites_encoded.append(site_text_encoder.encode(site_joined))

sites_padded = tf.keras.preprocessing.sequence.pad_sequences(sites_encoded,
                                                                 padding='post',maxlen=site_words)

sites_stacked = tf.stack(sites_padded)
sites_dataset = tf.data.Dataset.from_tensor_slices(
    (sites_stacked))

In [7]:

BUFFER_SIZE = 60000
BATCH_SIZE = 32
EPOCHS = 2
noise_dim = 100
# Batch and shuffle the data
train_dataset = sites_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [8]:
# Length of the vocabulary in chars
vocab_size = len(vocabulary_set)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 128
@tf.function
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

def build_generator():
  model = tf.keras.Sequential([

    tfl.Dense(site_words, use_bias=False, batch_input_shape=(None,noise_dim,)),
    tfl.ReLU(max_value=vocab_size-1),
    tfl.Embedding(site_words, embedding_dim),
    tfl.GRU(rnn_units,return_sequences=True,
                        recurrent_initializer='glorot_uniform'),
    tfl.Dense(vocab_size, activation='softmax'),
    tfl.Reshape((site_words,vocab_size),name="reshape_out"),
    #tfl.ReLU(max_value=vocab_size-1),
  ])
  return model

In [9]:
def make_discriminator_model():
    model = tf.keras.Sequential([
        tfl.Embedding(vocab_size, embedding_dim, batch_input_shape=(None,site_words,)),
        tfl.GRU(rnn_units,
                            recurrent_initializer='glorot_uniform'),

        tfl.Dense(vocab_size/4),
        tfl.Dense(vocab_size/16),
        tfl.Dense(1)
    ])

    return model


In [10]:
generator = build_generator()

In [11]:

discriminator = make_discriminator_model()

In [12]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 125)               12500     
_________________________________________________________________
re_lu (ReLU)                 (None, 125)               0         
_________________________________________________________________
embedding (Embedding)        (None, 125, 256)          32000     
_________________________________________________________________
gru (GRU)                    (None, 125, 128)          148224    
_________________________________________________________________
dense_1 (Dense)              (None, 125, 177)          22833     
_________________________________________________________________
reshape_out (Reshape)        (None, 125, 177)          0         
Total params: 215,557
Trainable params: 215,557
Non-trainable params: 0
__________________________________________________

In [13]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 125, 256)          45312     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               148224    
_________________________________________________________________
dense_2 (Dense)              (None, 44)                5676      
_________________________________________________________________
dense_3 (Dense)              (None, 11)                495       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 12        
Total params: 199,719
Trainable params: 199,719
Non-trainable params: 0
_________________________________________________________________


In [14]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
me=tf.keras.losses.MSE
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
generator_optimizer = tf.keras.optimizers.Adam(1e-3)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-3)

In [86]:
def softargmax(x, beta=1e10,axis=-1):
  x_range = tf.range(x.shape.as_list()[-1], dtype=x.dtype)
  return tf.reduce_sum(tf.nn.softmax(x*beta) * x_range, axis=axis)
def cleanSite(site):
  return tf.math.minimum(
    tf.ones_like(site)*vocab_size-1,
    site
)
@tf.function
def train_step(sites):
    sites=tf.cast(sites,tf.float32)
    noise = tf.random.normal([sites.shape[0], noise_dim])
    sites=cleanSite(sites)

    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:

      generated_sites= generator(noise, training=True)

      generated_sites_reduced=softargmax(
        generated_sites,
        axis=2
      )
      sites_clean=cleanSite(sites)
      generated_sites_reduced_clean=cleanSite(generated_sites_reduced)

      real_output = discriminator(sites_clean, training=True)
      fake_output = discriminator(generated_sites_reduced_clean, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      #gen_loss=me(sites,generated_sites_reduced_clean)

      gradients_of_generator = gen_tape.gradient(gen_loss,sites_clean )
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

      print("gen_err:",gen_loss)
      print("sites:",generated_sites,sites)
      print("gen_grads:",gradients_of_generator)

      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      
      print("--------------")

In [87]:
for epoch in range(EPOCHS):
  start = time.time()
  for site_batch in train_dataset:
    train_step(site_batch)
  print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


gen_err: Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
sites: Tensor("sequential/reshape_out/Reshape:0", shape=(32, 125, 177), dtype=float32) Tensor("Minimum:0", shape=(32, 125), dtype=float32)
gen_grads: None


TypeError: in converted code:

    <ipython-input-86-55724408c5a9>:41 train_step  *
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    /home/benedikt/my_py3/lib/python3.7/site-packages/tensorflow_core/python/autograph/impl/api.py:416 converted_call
        return py_builtins.overload_of(f)(*args)
    /home/benedikt/my_py3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/py_builtins.py:348 zip_
        return _py_zip(*iterables)
    /home/benedikt/my_py3/lib/python3.7/site-packages/tensorflow_core/python/autograph/operators/py_builtins.py:356 _py_zip
        return zip(*iterables)

    TypeError: zip argument #1 must support iteration


In [ ]:

noise = tf.random.normal([1, noise_dim])
script_data=generator(noise, training=False)

script_data_reduced=softargmax(
  script_data,
  axis=2
)
decoded=site_text_encoder.decode(tf.cast(script_data_reduced[0],tf.int32))
print("".join(decoded))

In [ ]:
print(vocab_size,script_data_reduced)